In [1]:
# Quick test to see if Python is working
print("🧪 Testing Python environment...")
import sys
print(f"Python version: {sys.version}")
print("✅ Python is working!")

🧪 Testing Python environment...
Python version: 3.11.13 (main, Aug 12 2025, 23:09:33) [GCC 14.2.0]
✅ Python is working!


In [2]:
# Test imports one by one to find the issue
print("Testing imports...")

# Basic Python modules
import os, json, re, uuid
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from datetime import datetime
print("✅ Basic Python modules imported")

# Test dotenv
try:
    from dotenv import load_dotenv
    print("✅ dotenv imported")
except ImportError as e:
    print(f"❌ dotenv failed: {e}")

# Test Azure
try:
    from azure.identity import DefaultAzureCredential, get_bearer_token_provider
    print("✅ Azure identity imported")
except ImportError as e:
    print(f"❌ Azure identity failed: {e}")

# Test LangChain
try:
    from langchain_openai import AzureChatOpenAI
    from langchain.prompts import ChatPromptTemplate
    from langchain.schema.output_parser import StrOutputParser
    print("✅ LangChain imported")
except ImportError as e:
    print(f"❌ LangChain failed: {e}")

# Test tiktoken
try:
    import tiktoken
    print("✅ tiktoken imported")
except ImportError as e:
    print(f"❌ tiktoken failed: {e}")

print("🎯 Import test complete!")

Testing imports...
✅ Basic Python modules imported
✅ dotenv imported
✅ Azure identity imported
✅ LangChain imported
✅ tiktoken imported
🎯 Import test complete!
✅ LangChain imported
✅ tiktoken imported
🎯 Import test complete!


# 📚 Progressive Book Summary Generator

## 🎯 **Objective**
Create comprehensive book summaries using progressive chapter-by-chapter analysis with GPT-5-mini.

## 🔄 **Progressive Summary Process**

```
Full Markdown Document
         ↓
📖 CHAPTER SPLITTING
   Chapter 1, Chapter 2, Chapter 3...
         ↓
🔄 SEQUENTIAL SUMMARIZATION
   Chapter 1 → Summary 1
   Chapter 2 + Summary 1 → Summary 2  
   Chapter 3 + Summary 1+2 → Summary 3
   ...
         ↓
📚 FINAL BOOK SUMMARY
   All Chapter Summaries → Complete Book Overview
```

## 🎓 **Benefits**
- **Progressive Context Building**: Each summary builds on previous knowledge
- **Natural Reading Flow**: Mirrors human document comprehension
- **Token Efficiency**: Manageable context windows at each step
- **Quality Summaries**: Deep understanding through sequential analysis

## 📋 Prerequisites

**Before running this notebook:**
1. Run `search_with_document_layout.ipynb` to generate markdown content
2. Ensure GPT-5-mini deployment is configured
3. Have markdown content ready for processing

In [3]:
# Import required libraries
import os
import json
import re
import uuid
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from datetime import datetime

# Azure and LangChain imports
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import tiktoken

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [4]:
# Load environment variables
load_dotenv(override=True)

# Azure OpenAI configs (GPT-5-mini)
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
AZURE_OPENAI_CHAT_API_VERSION = os.getenv("AZURE_OPENAI_CHAT_API_VERSION") or "2024-12-01-preview"

print(f"✅ Environment loaded")
print(f"🤖 Using GPT-5-mini deployment: {AZURE_OPENAI_CHAT_DEPLOYMENT_NAME}")

✅ Environment loaded
🤖 Using GPT-5-mini deployment: gpt-5-mini


In [5]:
# Initialize Azure clients
credential = DefaultAzureCredential()
azure_ad_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# Initialize GPT-5-mini client
chat_llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
    openai_api_version=AZURE_OPENAI_CHAT_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=azure_ad_token_provider,
    # temperature=1.0,  # GPT-5-mini only supports default temperature
    max_completion_tokens=4000   # Allow for detailed summaries (GPT-5-mini parameter)
)

print("✅ GPT-5-mini client initialized")
print("🧠 Context window: 272K input / 128K output")

/home/vscode/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: WARNING! max_completion_tokens is not default parameter.
                max_completion_tokens was transferred to model_kwargs.
                Please confirm that max_completion_tokens is what you intended.
  if await self.run_code(code, result, async_=asy):


✅ GPT-5-mini client initialized
🧠 Context window: 272K input / 128K output


## 📖 Chapter Splitting and Processing Classes

In [6]:
@dataclass
class BookChapter:
    """Represents a chapter from the document"""
    chapter_number: int
    title: str
    content: str
    token_count: int
    page_range: str
    
@dataclass
class ChapterSummary:
    """Represents a generated chapter summary"""
    chapter_number: int
    chapter_title: str
    summary: str
    key_concepts: List[str]
    main_topics: List[str]
    token_count: int
    created_at: datetime

@dataclass 
class BookSummary:
    """Represents the final comprehensive book summary"""
    book_title: str
    overall_summary: str
    chapter_summaries: List[ChapterSummary]
    key_themes: List[str]
    learning_objectives: List[str]
    total_chapters: int
    created_at: datetime

print("✅ Data structures defined")

✅ Data structures defined


In [7]:
class ProgressiveBookSummarizer:
    """Progressive book summarization using sequential chapter analysis"""
    
    def __init__(self, llm):
        self.llm = llm
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        self.chapter_summaries = []
        
        # Progressive chapter summary prompt
        self.chapter_summary_prompt = ChatPromptTemplate.from_template("""
You are an expert book summarizer creating progressive chapter summaries.

CONTEXT FROM PREVIOUS CHAPTERS:
{previous_context}

CURRENT CHAPTER TO SUMMARIZE:
Chapter: {chapter_number}
Title: {chapter_title}
Content:
{chapter_content}

Create a comprehensive summary that:
1. **Builds on previous chapters** - Reference relevant concepts from earlier chapters
2. **Identifies key concepts** - Extract main ideas and important terms
3. **Explains relationships** - Show how this chapter connects to previous content
4. **Highlights progression** - Show how understanding is building

Format as JSON:
{{
    "chapter_summary": "Detailed 2-3 paragraph summary building on previous chapters",
    "key_concepts": ["concept1", "concept2", "concept3"],
    "main_topics": ["topic1", "topic2", "topic3"],
    "connections_to_previous": "How this chapter relates to previous chapters",
    "new_insights": "What new understanding this chapter provides"
}}
""")
        
        # Final book summary prompt
        self.book_summary_prompt = ChatPromptTemplate.from_template("""
You are an expert book summarizer creating a comprehensive book overview.

ALL CHAPTER SUMMARIES:
{all_chapter_summaries}

Create a comprehensive book summary that:
1. **Overall narrative** - Tell the complete story of the book
2. **Key themes** - Identify major themes running through the book
3. **Learning progression** - Show how concepts build throughout
4. **Practical value** - What readers will gain from this book

Format as JSON:
{{
    "book_title": "Inferred or provided book title",
    "overall_summary": "Comprehensive 4-5 paragraph book summary",
    "key_themes": ["theme1", "theme2", "theme3"],
    "learning_objectives": ["objective1", "objective2", "objective3"],
    "book_structure": "How the book is organized and flows",
    "target_audience": "Who would benefit from this book",
    "practical_applications": "How readers can apply this knowledge"
}}
""")
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text"""
        return len(self.tokenizer.encode(text))
    
    def split_into_chapters(self, markdown_content: str) -> List[BookChapter]:
        """Split markdown content into logical chapters"""
        print("📖 Splitting document into chapters...")
        
        # Split by markdown headers (# ## ###)
        # This is a smart splitter that looks for natural chapter boundaries
        chapter_pattern = r'^(#{1,3})\s+(.+?)$'
        sections = re.split(chapter_pattern, markdown_content, flags=re.MULTILINE)
        
        chapters = []
        current_chapter = None
        chapter_number = 0
        
        i = 0
        while i < len(sections):
            if i + 2 < len(sections) and sections[i+1]:  # Found a header
                header_level = sections[i+1]
                title = sections[i+2].strip()
                content = sections[i+3] if i+3 < len(sections) else ""
                
                # Only treat as new chapter if it's a major header (# or ##)
                if len(header_level) <= 2:  # # or ##
                    if current_chapter:  # Save previous chapter
                        chapters.append(current_chapter)
                    
                    chapter_number += 1
                    current_chapter = BookChapter(
                        chapter_number=chapter_number,
                        title=title,
                        content=content.strip(),
                        token_count=self.count_tokens(content),
                        page_range=f"Chapter {chapter_number}"
                    )
                else:  # Subsection, add to current chapter
                    if current_chapter:
                        current_chapter.content += f"\n\n{header_level} {title}\n{content}"
                        current_chapter.token_count = self.count_tokens(current_chapter.content)
                
                i += 4
            else:
                i += 1
        
        # Add the last chapter
        if current_chapter:
            chapters.append(current_chapter)
        
        # If no clear chapters found, create artificial chapters by content length
        if not chapters:
            print("⚠️  No clear chapter structure found, creating artificial chapters...")
            chapters = self._create_artificial_chapters(markdown_content)
        
        print(f"✅ Found {len(chapters)} chapters")
        for i, chapter in enumerate(chapters, 1):
            print(f"   Chapter {i}: {chapter.title[:50]}... ({chapter.token_count:,} tokens)")
        
        return chapters
    
    def _create_artificial_chapters(self, content: str, target_tokens: int = 15000) -> List[BookChapter]:
        """Create artificial chapters based on content length"""
        words = content.split()
        chapters = []
        chapter_number = 0
        
        # Estimate words per chapter (roughly 4 tokens per word)
        words_per_chapter = target_tokens // 4
        
        for i in range(0, len(words), words_per_chapter):
            chapter_number += 1
            chapter_words = words[i:i + words_per_chapter]
            chapter_content = " ".join(chapter_words)
            
            # Try to find a good title from the first few sentences
            first_sentences = chapter_content[:200].split('. ')
            title = f"Section {chapter_number}: {first_sentences[0][:50]}..."
            
            chapters.append(BookChapter(
                chapter_number=chapter_number,
                title=title,
                content=chapter_content,
                token_count=self.count_tokens(chapter_content),
                page_range=f"Section {chapter_number}"
            ))
        
        return chapters

print("✅ ProgressiveBookSummarizer class defined")

✅ ProgressiveBookSummarizer class defined


## 🔄 Progressive Summarization Methods

In [8]:
class ProgressiveBookSummarizer(ProgressiveBookSummarizer):
    """Extended class with progressive summarization methods"""
    
    def summarize_chapter_progressively(self, chapter: BookChapter, previous_summaries: List[ChapterSummary]) -> ChapterSummary:
        """Summarize a chapter with context from previous chapters"""
        
        print(f"📝 Summarizing Chapter {chapter.chapter_number}: {chapter.title[:50]}...")
        
        # Build context from previous summaries
        previous_context = ""
        if previous_summaries:
            context_parts = []
            for prev_summary in previous_summaries:
                context_parts.append(f"**Chapter {prev_summary.chapter_number}**: {prev_summary.summary}")
            previous_context = "\n\n".join(context_parts)
        else:
            previous_context = "This is the first chapter - no previous context available."
        
        # Generate summary with progressive context
        summary_chain = self.chapter_summary_prompt | self.llm | StrOutputParser()
        
        response = summary_chain.invoke({
            "previous_context": previous_context,
            "chapter_number": chapter.chapter_number,
            "chapter_title": chapter.title,
            "chapter_content": chapter.content
        })
        
        try:
            summary_data = json.loads(response)
            
            chapter_summary = ChapterSummary(
                chapter_number=chapter.chapter_number,
                chapter_title=chapter.title,
                summary=summary_data.get("chapter_summary", ""),
                key_concepts=summary_data.get("key_concepts", []),
                main_topics=summary_data.get("main_topics", []),
                token_count=self.count_tokens(summary_data.get("chapter_summary", "")),
                created_at=datetime.now()
            )
            
            print(f"✅ Chapter {chapter.chapter_number} summarized ({chapter_summary.token_count} tokens)")
            print(f"   Key concepts: {', '.join(chapter_summary.key_concepts[:3])}...")
            
            return chapter_summary
            
        except json.JSONDecodeError as e:
            print(f"⚠️  JSON parsing failed for Chapter {chapter.chapter_number}, using raw response")
            return ChapterSummary(
                chapter_number=chapter.chapter_number,
                chapter_title=chapter.title,
                summary=response,
                key_concepts=[],
                main_topics=[],
                token_count=self.count_tokens(response),
                created_at=datetime.now()
            )
    
    def create_final_book_summary(self, chapter_summaries: List[ChapterSummary]) -> BookSummary:
        """Create comprehensive book summary from all chapter summaries"""
        
        print("📚 Creating final book summary from all chapters...")
        
        # Compile all chapter summaries
        all_summaries_text = "\n\n".join([
            f"**Chapter {cs.chapter_number}: {cs.chapter_title}**\n{cs.summary}\nKey Concepts: {', '.join(cs.key_concepts)}"
            for cs in chapter_summaries
        ])
        
        # Generate final book summary
        book_summary_chain = self.book_summary_prompt | self.llm | StrOutputParser()
        
        response = book_summary_chain.invoke({
            "all_chapter_summaries": all_summaries_text
        })
        
        try:
            book_data = json.loads(response)
            
            book_summary = BookSummary(
                book_title=book_data.get("book_title", "Document Summary"),
                overall_summary=book_data.get("overall_summary", ""),
                chapter_summaries=chapter_summaries,
                key_themes=book_data.get("key_themes", []),
                learning_objectives=book_data.get("learning_objectives", []),
                total_chapters=len(chapter_summaries),
                created_at=datetime.now()
            )
            
            print(f"✅ Book summary created: {book_summary.book_title}")
            print(f"   Total chapters: {book_summary.total_chapters}")
            print(f"   Key themes: {len(book_summary.key_themes)}")
            
            return book_summary
            
        except json.JSONDecodeError as e:
            print(f"⚠️  JSON parsing failed for book summary, using raw response")
            return BookSummary(
                book_title="Document Summary",
                overall_summary=response,
                chapter_summaries=chapter_summaries,
                key_themes=[],
                learning_objectives=[],
                total_chapters=len(chapter_summaries),
                created_at=datetime.now()
            )
    
    def process_document_progressively(self, markdown_content: str) -> BookSummary:
        """Complete progressive book summarization process"""
        
        print("🚀 Starting Progressive Book Summarization")
        print("=" * 60)
        
        # Step 1: Split into chapters
        chapters = self.split_into_chapters(markdown_content)
        
        # Step 2: Progressive chapter summarization
        print("\n📝 Progressive Chapter Summarization")
        print("-" * 40)
        
        chapter_summaries = []
        
        for chapter in chapters:
            # Summarize with context of all previous summaries
            chapter_summary = self.summarize_chapter_progressively(chapter, chapter_summaries)
            chapter_summaries.append(chapter_summary)
            
            print(f"   Context now includes {len(chapter_summaries)} chapters")
        
        # Step 3: Create final book summary
        print("\n📚 Final Book Summary Generation")
        print("-" * 40)
        
        book_summary = self.create_final_book_summary(chapter_summaries)
        
        print("\n🎯 Progressive Summarization Complete!")
        print("=" * 60)
        
        return book_summary

print("✅ Progressive summarization methods added")

✅ Progressive summarization methods added


## 🧪 Testing and Demo Section

In [9]:
# Initialize the progressive book summarizer
print("🚀 Initializing Progressive Book Summarizer...")

summarizer = ProgressiveBookSummarizer(chat_llm)

print("✅ Progressive Book Summarizer ready!")
print("\n💡 Usage:")
print("   # Load your markdown content from Content Understanding")
print("   # markdown_content = load_your_markdown_here()")
print("   # book_summary = summarizer.process_document_progressively(markdown_content)")
print("\n🎯 This will create progressive chapter summaries and a final book summary")

🚀 Initializing Progressive Book Summarizer...
✅ Progressive Book Summarizer ready!

💡 Usage:
   # Load your markdown content from Content Understanding
   # markdown_content = load_your_markdown_here()
   # book_summary = summarizer.process_document_progressively(markdown_content)

🎯 This will create progressive chapter summaries and a final book summary
✅ Progressive Book Summarizer ready!

💡 Usage:
   # Load your markdown content from Content Understanding
   # markdown_content = load_your_markdown_here()
   # book_summary = summarizer.process_document_progressively(markdown_content)

🎯 This will create progressive chapter summaries and a final book summary


## 💾 Save Results

In [10]:
# Save book summary to file
def save_book_summary(book_summary: BookSummary, output_path: str = "../educational_content/book_summaries", custom_filename: str = None):
    """Save book summary to JSON file"""
    
    # Create output directory
    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create filename with timestamp or use custom filename
    if custom_filename:
        filename = custom_filename
    else:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"book_summary_{timestamp}.json"
    
    filepath = output_dir / filename
    
    # Convert to dictionary for JSON serialization
    summary_dict = {
        "book_title": book_summary.book_title,
        "overall_summary": book_summary.overall_summary,
        "key_themes": book_summary.key_themes,
        "learning_objectives": book_summary.learning_objectives,
        "total_chapters": book_summary.total_chapters,
        "created_at": book_summary.created_at.isoformat(),
        "chapter_summaries": [
            {
                "chapter_number": cs.chapter_number,
                "chapter_title": cs.chapter_title,
                "summary": cs.summary,
                "key_concepts": cs.key_concepts,
                "main_topics": cs.main_topics,
                "token_count": cs.token_count,
                "created_at": cs.created_at.isoformat()
            } for cs in book_summary.chapter_summaries
        ]
    }
    
    # Save to file
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(summary_dict, f, indent=2, ensure_ascii=False)
    
    print(f"💾 Book summary saved to: {filepath}")
    return filepath

print("✅ Save function ready!")
print("💡 Usage: save_book_summary(your_book_summary, custom_filename='custom_name.json')")

✅ Save function ready!
💡 Usage: save_book_summary(your_book_summary, custom_filename='custom_name.json')


In [11]:
# 🚀 REAL DOCUMENT TEST: Process Venture Deals with Progressive Summarization
print("📚 PROGRESSIVE BOOK SUMMARIZATION - REAL DOCUMENT TEST")
print("=" * 80)

# Load the real Venture Deals markdown content
venture_deals_path = "../educational_content/venture_deals_markdown.md"
print(f"📄 Loading: {venture_deals_path}")

try:
    with open(venture_deals_path, 'r', encoding='utf-8') as f:
        venture_deals_content = f.read()
    
    print(f"✅ Content loaded successfully!")
    print(f"📊 Content statistics:")
    print(f"   📝 Characters: {len(venture_deals_content):,}")
    print(f"   📄 Lines: {venture_deals_content.count(chr(10)) + 1:,}")
    print(f"   🧮 Estimated tokens: {len(venture_deals_content.split()) * 1.3:.0f}")
    
    # Show first few lines as preview
    first_lines = venture_deals_content.split('\n')[:10]
    print(f"\\n📖 Content preview (first 10 lines):")
    print("-" * 50)
    for i, line in enumerate(first_lines, 1):
        print(f"{i:2d}: {line[:80]}...")
    
    print("\\n🎯 Ready to run Progressive Book Summarization!")
    print("⚠️  Warning: This is a large document (~135K tokens) - processing may take 10-15 minutes")
    
except FileNotFoundError:
    print(f"❌ File not found: {venture_deals_path}")
    print("Please make sure the Content Understanding pipeline has been run first.")
except Exception as e:
    print(f"❌ Error loading file: {e}")

📚 PROGRESSIVE BOOK SUMMARIZATION - REAL DOCUMENT TEST
📄 Loading: ../educational_content/venture_deals_markdown.md
❌ File not found: ../educational_content/venture_deals_markdown.md
Please make sure the Content Understanding pipeline has been run first.


In [12]:
# 🚀 RUN Progressive Summarization on Venture Deals
print("🔥 STARTING PROGRESSIVE SUMMARIZATION - VENTURE DEALS")
print("=" * 80)
print("📚 Processing: 'Venture Deals' by Brad Feld & Jason Mendelson")
print("⏱️  Estimated time: 10-15 minutes for ~135K tokens")
print("🧠 Using GPT-5-mini with 272K context window")
print("")

# Start timing
import time
start_time = time.time()

try:
    # Run the progressive book summarization
    print("🚀 Beginning progressive summarization...")
    venture_deals_summary = summarizer.process_document_progressively(venture_deals_content)
    
    # Calculate processing time
    end_time = time.time()
    processing_time = end_time - start_time
    
    print("\\n🎉 PROGRESSIVE SUMMARIZATION COMPLETE!")
    print("=" * 80)
    print(f"⏱️  Total processing time: {processing_time/60:.1f} minutes")
    print(f"📚 Book title: {venture_deals_summary.book_title}")
    print(f"📖 Total chapters: {venture_deals_summary.total_chapters}")
    print(f"🎯 Key themes: {len(venture_deals_summary.key_themes)}")
    print(f"🎓 Learning objectives: {len(venture_deals_summary.learning_objectives)}")
    
    # Show summary preview
    print(f"\\n📄 Book Summary Preview:")
    print("-" * 50)
    print(venture_deals_summary.overall_summary[:500] + "...")
    
    print(f"\\n🔑 Key Themes:")
    for i, theme in enumerate(venture_deals_summary.key_themes[:5], 1):
        print(f"   {i}. {theme}")
    if len(venture_deals_summary.key_themes) > 5:
        print(f"   ... and {len(venture_deals_summary.key_themes) - 5} more")
    
    print("\\n✅ Ready to save comprehensive book summary!")
    
except Exception as e:
    print(f"❌ Error during processing: {e}")
    import traceback
    traceback.print_exc()

🔥 STARTING PROGRESSIVE SUMMARIZATION - VENTURE DEALS
📚 Processing: 'Venture Deals' by Brad Feld & Jason Mendelson
⏱️  Estimated time: 10-15 minutes for ~135K tokens
🧠 Using GPT-5-mini with 272K context window

🚀 Beginning progressive summarization...
❌ Error during processing: name 'venture_deals_content' is not defined


Traceback (most recent call last):
  File "/tmp/ipykernel_27596/3581662100.py", line 16, in <module>
    venture_deals_summary = summarizer.process_document_progressively(venture_deals_content)
                                                                      ^^^^^^^^^^^^^^^^^^^^^
NameError: name 'venture_deals_content' is not defined


In [13]:
# 💾 Save the Venture Deals Book Summary
print("💾 SAVING VENTURE DEALS BOOK SUMMARY")
print("=" * 60)

if 'venture_deals_summary' in locals():
    # Save the comprehensive book summary with custom filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"venture_deals_book_summary_{timestamp}.json"
    filepath = save_book_summary(venture_deals_summary, "../educational_content/book_summaries", filename)
    
    print(f"\n📚 VENTURE DEALS SUMMARY ANALYSIS")
    print("=" * 50)
    print(f"📖 Book Title: {venture_deals_summary.book_title}")
    print(f"📄 Total Chapters: {venture_deals_summary.total_chapters}")
    print(f"🎯 Key Themes: {len(venture_deals_summary.key_themes)}")
    print(f"🎓 Learning Objectives: {len(venture_deals_summary.learning_objectives)}")
    
    print(f"\n🔑 Key Themes Identified:")
    for i, theme in enumerate(venture_deals_summary.key_themes, 1):
        print(f"   {i:2d}. {theme}")
    
    print(f"\n🎓 Learning Objectives:")
    for i, objective in enumerate(venture_deals_summary.learning_objectives, 1):
        print(f"   {i:2d}. {objective}")
    
    print(f"\n📋 Chapter Breakdown:")
    for i, chapter_summary in enumerate(venture_deals_summary.chapter_summaries[:5], 1):
        print(f"   Chapter {chapter_summary.chapter_number}: {chapter_summary.chapter_title[:60]}...")
        print(f"      Key concepts: {', '.join(chapter_summary.key_concepts[:3])}...")
    
    if len(venture_deals_summary.chapter_summaries) > 5:
        print(f"   ... and {len(venture_deals_summary.chapter_summaries) - 5} more chapters")
    
    print(f"\n📄 Book Summary (first 300 characters):")
    print("-" * 40)
    print(venture_deals_summary.overall_summary[:300] + "...")
    
    print(f"\n🎉 SUCCESS! Complete progressive book summary generated and saved!")
    print(f"💾 Saved to: {filepath}")
else:
    print("❌ venture_deals_summary not found - processing may have failed")

💾 SAVING VENTURE DEALS BOOK SUMMARY
❌ venture_deals_summary not found - processing may have failed


## 🚀 How to Use the New Production Summarizer

Now that we have a production-ready implementation, you have several options to summarize markdown content:

### 🎯 Option 1: Use the Production Implementation Directly

The recommended approach is to use our new production code that works exactly like this notebook:

```python
# Import the production summarizer
from src.summarization.book_summarizer import ProgressiveBookSummarizer

# Initialize (automatically handles Azure OpenAI setup)
summarizer = ProgressiveBookSummarizer()

# Process your markdown content
result = summarizer.process_document_progressively(
    markdown_content=your_markdown_content,
    book_title="Your Book Title"
)

# Access results
print(f"Book Title: {result.book_title}")
print(f"Summary: {result.overall_summary}")
print(f"Key Themes: {result.key_themes}")
print(f"Learning Objectives: {result.learning_objectives}")
```

### 🌐 Option 2: Use the REST API

Start the API server and use HTTP endpoints:

```bash
# Start the server
python run_server.py

# Option 2a: Send content directly
curl -X POST http://localhost:8000/summarization/summarize \
  -H "Content-Type: application/json" \
  -d '{
    "book_title": "My Book", 
    "markdown_content": "# Chapter 1\nYour content here..."
  }'

# Option 2b: Upload a markdown file
curl -X POST http://localhost:8000/summarization/summarize-file \
  -F "file=@your_book.md" \
  -F "book_title=My Book"
```

### 🧪 Option 3: Continue Using This Notebook

You can still use this notebook implementation - both work identically!

In [ ]:
# 🎯 PRACTICAL EXAMPLE: Using the New Production Summarizer
print("🚀 USING THE NEW PRODUCTION SUMMARIZER")
print("=" * 60)

# Method 1: Import and use the production implementation directly
try:
    # Add the src directory to Python path
    import sys
    from pathlib import Path
    sys.path.insert(0, str(Path("..").resolve()))
    
    # Import the production summarizer
    from src.summarization.book_summarizer import ProgressiveBookSummarizer
    
    print("✅ Production summarizer imported successfully!")
    
    # Create a test markdown example
    test_markdown = """# Introduction to Machine Learning

Machine learning is a subset of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed for every task.

## What is Machine Learning?

At its core, machine learning involves training algorithms on data to recognize patterns and make predictions or decisions on new, unseen data.

### Types of Machine Learning

1. **Supervised Learning**: Learning with labeled examples
2. **Unsupervised Learning**: Finding patterns in unlabeled data  
3. **Reinforcement Learning**: Learning through trial and error

## Applications

Machine learning powers many technologies we use daily:
- Recommendation systems (Netflix, Amazon)
- Image recognition (photo tagging)
- Natural language processing (chatbots)
- Autonomous vehicles

# Getting Started

To begin your machine learning journey, you'll need to understand:
- Basic statistics and probability
- Programming (Python/R)
- Data manipulation and visualization
- Algorithm fundamentals

## Conclusion

Machine learning is transforming industries and creating new possibilities for solving complex problems through data-driven insights.
"""

    # Initialize the production summarizer (it handles Azure OpenAI setup automatically)
    prod_summarizer = ProgressiveBookSummarizer()
    
    print("🧪 Testing with sample markdown content...")
    print(f"📄 Content length: {len(test_markdown)} characters")
    
    # Process the markdown content
    result = prod_summarizer.process_document_progressively(
        markdown_content=test_markdown,
        book_title="Introduction to Machine Learning"
    )
    
    print("\n✅ PRODUCTION SUMMARIZER RESULTS:")
    print("=" * 50)
    print(f"📖 Book Title: {result.book_title}")
    print(f"📚 Total Chapters: {result.total_chapters}")
    print(f"🎯 Key Themes ({len(result.key_themes)}):")
    for i, theme in enumerate(result.key_themes, 1):
        print(f"   {i}. {theme}")
    
    print(f"\n📝 Summary (first 200 chars):")
    print(result.overall_summary[:200] + "...")
    
    print("\n🎉 SUCCESS! The production summarizer works perfectly!")
    print("💡 You can now use this production code anywhere in your project!")
    
except Exception as e:
    print(f"⚠️  Production summarizer not available: {e}")
    print("💡 You can still use the notebook implementation above!")
    print("🔧 To fix this, make sure you've run the migration setup.")

## 📚 Quick Reference: Summarizing Your Markdown

### For Your Specific Use Case:

1. **If you have a markdown file ready:**
   ```python
   # Load your markdown content
   with open("path/to/your/book.md", "r", encoding="utf-8") as f:
       markdown_content = f.read()
   
   # Summarize it
   from src.summarization.book_summarizer import ProgressiveBookSummarizer
   summarizer = ProgressiveBookSummarizer()
   result = summarizer.process_document_progressively(markdown_content, "Your Book Title")
   ```

2. **If you want to use the API:**
   ```bash
   # Start server in terminal: python run_server.py
   # Then upload your file:
   curl -X POST http://localhost:8000/summarization/summarize-file \
     -F "file=@your_book.md" -F "book_title=Your Book"
   ```

3. **If you want to continue with this notebook:**
   - Just run the cells above to use the notebook implementation
   - Both implementations work identically!

### 🎯 The production summarizer gives you:
- ✅ Progressive chapter-by-chapter analysis
- ✅ Comprehensive book summaries with themes and objectives  
- ✅ JSON-structured responses
- ✅ Ready for deployment and scaling
- ✅ Works exactly like this notebook